In [2]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import torch
import evaluate
import wandb
import random
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from google.colab import userdata # hf token import from secrets

In [7]:
wandb.init(mode='disabled')
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
transformers.set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [9]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [11]:
def distilbert(text_column, repository_id):
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    def preprocess_function(examples):
        return tokenizer(examples[text_column], truncation=True)

    # data
    columns = [text_column] + ['label']
    df_train = pd.read_csv("train.csv")
    df_train = df_train[columns]

    df_test = pd.read_csv("test.csv")
    df_test = df_test[columns]

    train_dataset = Dataset.from_pandas(df_train).train_test_split(test_size=0.15)
    test_dataset = Dataset.from_pandas(df_test)

    print(train_dataset)
    print(test_dataset)

    tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
    tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

    # load model
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
    )

    model = model.to(device)
    print(model)

    training_args = TrainingArguments(
        output_dir="my_awesome_model",
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        eval_steps=20,
        logging_steps=20,
        weight_decay=0.01,
        eval_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        report_to="tensorboard",
        push_to_hub=True,
        hub_strategy="every_save",
        hub_model_id=repository_id,
        hub_token=userdata.get('hf'),
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset['train'],
        eval_dataset=tokenized_train_dataset['test'],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer.evaluate(eval_dataset=tokenized_test_dataset)

### distilbert for unmasked

In [12]:
print(distilbert(
    text_column='candidate',
    repository_id = "distilbert-base-uncased-0.0.1"
))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['candidate', 'label'],
        num_rows: 3455
    })
    test: Dataset({
        features: ['candidate', 'label'],
        num_rows: 610
    })
})
Dataset({
    features: ['candidate', 'label'],
    num_rows: 1718
})


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


<ipython-input-11-032c1d0ffb55>:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
20,0.664700,0.638682,0.626230
40,0.589900,0.573101,0.686885
60,0.529200,0.525568,0.711475
80,0.485700,0.522061,0.714754
100,0.484900,0.517077,0.727869
120,0.449500,0.495654,0.737705
140,0.398400,0.473347,0.750820
160,0.415300,0.489255,0.752459
180,0.427200,0.466593,0.744262
200,0.377900,0.470190,0.768852


{'eval_loss': 0.4886702001094818, 'eval_accuracy': 0.7770663562281723, 'eval_runtime': 1.8129, 'eval_samples_per_second': 947.67, 'eval_steps_per_second': 29.787, 'epoch': 3.0}


### distilbert for masked

In [13]:
print(distilbert(
    text_column='candidate masked',
    repository_id = "distilbert-base-uncased-0.0.1"
))

DatasetDict({
    train: Dataset({
        features: ['candidate masked', 'label'],
        num_rows: 3455
    })
    test: Dataset({
        features: ['candidate masked', 'label'],
        num_rows: 610
    })
})
Dataset({
    features: ['candidate masked', 'label'],
    num_rows: 1718
})


Map:   0%|          | 0/3455 [00:00<?, ? examples/s]

Map:   0%|          | 0/610 [00:00<?, ? examples/s]

Map:   0%|          | 0/1718 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


<ipython-input-11-032c1d0ffb55>:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
20,0.660900,0.628467,0.626230
40,0.581700,0.559814,0.704918
60,0.550500,0.536554,0.726230
80,0.514200,0.530576,0.721311
100,0.511300,0.529294,0.722951
120,0.451600,0.498706,0.739344
140,0.409700,0.478102,0.767213
160,0.428400,0.517635,0.711475
180,0.448600,0.482840,0.760656
200,0.393000,0.480264,0.762295


{'eval_loss': 0.4758956730365753, 'eval_accuracy': 0.779976717112922, 'eval_runtime': 1.853, 'eval_samples_per_second': 927.127, 'eval_steps_per_second': 29.141, 'epoch': 3.0}
